In [1]:
import transformers
MODEL_NAME = "results/checkpoint-3000"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
!optimum-cli export onnx --task text2text-generation-with-past --model {MODEL_NAME} {EXPORT_PATH}

/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Framework not specified. Using pt to export the model.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated wo

In [4]:
! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/spiece.model

In [6]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3


In [7]:
!pip install --upgrade pyspark
!pip install --upgrade spark-nlp

  Using cached pyspark-3.5.1-py2.py3-none-any.whl
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.2.3
    Uninstalling pyspark-3.2.3:
      Successfully uninstalled pyspark-3.2.3


In [8]:
import sparknlp
spark = sparknlp.start()

:: loading settings :: url = jar:file:/opt/module/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-83d21c82-4663-4cd7-9511-4b5465c99eca;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.g

In [11]:
import time
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
import json
import threading
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType
import sparknlp
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import T5Transformer
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *

consumer = KafkaConsumer(
    'SparkNLPKafka',
    bootstrap_servers='localhost:9092',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

T5 = T5Transformer.loadSavedModel(EXPORT_PATH, spark)\
  .setUseCache(True) \
  .setTask("summarize:") \
  .setMaxOutputLength(200)
T5.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

# 创建Spark-NLP模型Pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

T5 = T5Transformer.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("summary")
# 创建管道

pipeline = Pipeline().setStages([document_assembler, T5])

# 定义处理函数
def process_message(message):
    question = message['question']
    context = message['context']
    combined_text = f"Please answer to this question: {question} Context: {context}"
    return combined_text, question
# 创建Kafka消费数据的函数
def kafka_streaming():
    start_time = time.time()
    for msg in consumer:
        current_time = time.time()
        if current_time - start_time > 20:
            break
        message = msg.value
        combined_text, question = process_message(message)
        data = spark.createDataFrame([(Q,)], ["text"])
        result = pipeline.fit(data).transform(data)
        summaries = result.select("summary.result").collect()
        for summary in summaries:
            print(f"Question: {question}")
            print(f"Answer: {summary['result'][0]}")
# 使用线程运行Kafka消费者
thread = threading.Thread(target=kafka_streaming)
thread.start()

Using CPUs
Using CPUs
Using CPUs
Using CPUs


Question: What is the foundation for separation results within complexity classes?
Answer: a reversal


Question: What is responsible for constraining P according to the time hierarchy theorem?
Answer: a reversal


Question: Within what variable is L constrained according to the space hierarchy theorem?
Answer: a definite reversal


Question: What does not form the basis for most separation results of complexity classes?
Answer: a time and space hierarchy theorem


Question: What does the past time and space hierarchy theorems form the basis of?
Answer: the theory of the universe


Question: What is not strictly contained in EXPTIME?
Answer: a definite reversal


Question: What is not strictly contained in PSPACE?
Answer: a definite reversal


Question: What concept is frequently used to define complexity classes?
Answer: a reduction


Question: Reduction essentially takes one problem and converts into what?
Answer: a solution
